In [1]:
from keras.models import load_model
import numpy as np
import pandas as pd
import tensorflow as tf
from geopy import distance
#!pip install geopandas
import geopandas
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

import keras 
from keras.models import Sequential
from keras.layers import Dense, Dropout
import tensorflow as tf
from keras.utils import np_utils

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers
#from tensorflow.keras import Sequential
#from tensorflow.keras import Dense, Dropout
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, InputLayer, Reshape
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from keras import metrics

from keras.utils import np_utils

import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
import seaborn as sns

In [7]:
#Pipeline
def distance_km(x):
    home = (x['lat'], x['long'])
    merch = (x['merch_lat'], x['merch_long'])
    return distance.distance(home, merch).km

def NeuralNetworkPredict(url):
    model = load_model('model')
    df_pred = pd.read_csv(url)
    df_pred.drop(columns={"Unnamed: 0",'is_fraud'}, inplace=True)

    df_pred['distance_km'] = df_pred.apply(lambda x: distance_km(x), axis=1)
    df_pred['trans_date_trans_time'] = pd.to_datetime(df_pred['trans_date_trans_time'])
    df_pred['dob'] = pd.to_datetime(df_pred['dob'])

    df_pred['age_transaction'] = df_pred.apply(lambda x: (x['trans_date_trans_time'] - x['dob']).days // 365, axis=1)

    df_pred.drop(columns=['trans_date_trans_time','cc_num','merchant','first','last','street','lat','long','dob','trans_num','merch_lat','merch_long','job','city'],inplace=True)
    df_pred = pd.get_dummies(df_pred,drop_first=True)

    y_pred = model.predict(df_pred)
    print(y_pred, y_pred.value_counts())


In [9]:
#Input url or file name and run into function and run.
NeuralNetworkPredict("https://jrssbcrsefilesnait.blob.core.windows.net/3950data1/fraudTrain.csv.zip")

40522/40522 [==============================] - 40s 984us/step
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


### What I Did:

1. Calculated the distance between home and merchant using the latitude and longitude coordinates and created an new feature called 'distance_km'
2. Converted 'trans_date_trans_time' and 'dob' into datetime datatype, then calculated the age of the person during the time of the transaction, new feature is created called 'age_transaction'
3. Checked target balance, target was very imbalanced, which was expected for fraud transactions
4. Dropped the features that were logically useless and likely had no correlation with fraud, also dropped features used to calculate distance_km and age_transaction
5. To deal with imbalance, I calculated the log ratio of the frauds and non-frauds, and used this ratio as the bias_initializer parameter in the output layer of my neural network. This lets the neural network model deal with the imbalance in the data as it is training. 
6. Used standard scalar to normalize numerical data and one-hot encoding to encode categorical data
7. Split data for testing and training, used neural network to train model
8. Added dropout layer between dense layers of neural network to prevent overfitting. 
9. Saved model, loaded model in new .ipynb file
10. Created function to used the trained model on new data to make predictions